In [6]:
import pandas as pd
import numpy as np

# ==============================================================================
# SCRIPT DE GENERACIÓN DE DATOS SINTÉTICOS DE CONTEXTO (DATA AUGMENTATION)
# PROPOSITO: Generar un dataset histórico de variables socioeconómicas 
# ajustado a la realidad demográfica y eventos económicos (COVID-19).
# ==============================================================================

# ------------------------------------------------------------------------------
# 1. EXTRACCIÓN DE ENTIDADES GEOGRÁFICAS (FUENTE DE VERDAD)
# ------------------------------------------------------------------------------
# Objetivo: Identificar qué provincias existen realmente en el dataset principal de crímenes
# para asegurar que los datos generados coincidan exactamente (Integridad Referencial).
try:
    # Se carga el archivo maestro de delitos para extraer el catálogo de provincias
    df_base = pd.read_excel('C:/Users/Oliver/Downloads/demo1/src/Dataset_Homicidios_Panama_2017_2024_NormalizadoFINAL.xlsx')
    
    # Se estandarizan los nombres (Mayúsculas y sin espacios extra) para evitar errores de duplicidad
    provincias_reales = df_base['PROVINCIA'].astype(str).str.upper().str.strip().unique()
    
    print(f"[INFO] Entidades geográficas detectadas en el origen: {len(provincias_reales)}")
    print(provincias_reales)
except Exception as e:
    print(f"[ERROR] Error en la lectura del archivo fuente: {e}")
    # En caso de fallo de lectura, se utiliza una lista de respaldo estandarizada
    provincias_reales = [
        'BOCAS DEL TORO', 'COCLÉ', 'COLÓN', 'CHIRIQUÍ', 'DARIÉN', 'HERRERA', 
        'LOS SANTOS', 'PANAMÁ', 'PANAMÁ OESTE', 'VERAGUAS', 'SAN MIGUELITO',
        'GUNA YALA', 'EMBERÁ', 'NGÄBE BUGLÉ'
    ]

# ------------------------------------------------------------------------------
# 2. PARAMETRIZACIÓN BASE (BASE DE CONOCIMIENTO)
# ------------------------------------------------------------------------------
# Se define un diccionario con valores estadísticos iniciales basados en fuentes oficiales
# (INEC / MinSeg) para el año base 2023.
# Variables:
# - pop: Población estimada.
# - unemp: Tasa de Desempleo (%).
# - gang: Índice de actividad de pandillas (Escala 1-10).

datos_maestros = {
    # GRUPO A: Zonas de Alta Densidad Urbana y Riesgo Social
    "PANAMÁ":        {"pop": 1750000, "unemp": 8.5,  "gang": 8},
    "SAN MIGUELITO": {"pop": 380000,  "unemp": 9.5,  "gang": 10}, # Zona de alta incidencia delictiva
    "COLÓN":         {"pop": 295000,  "unemp": 11.5, "gang": 10}, # Alto desempleo estructural
    "PANAMÁ OESTE":  {"pop": 610000,  "unemp": 10.5, "gang": 7},  # Zona de expansión demográfica rápida

    # GRUPO B: Zonas Productivas / Riesgo Medio
    "CHIRIQUÍ":      {"pop": 475000,  "unemp": 6.5,  "gang": 5},
    "BOCAS DEL TORO":{"pop": 185000,  "unemp": 7.0,  "gang": 4},
    "VERAGUAS":      {"pop": 255000,  "unemp": 4.5,  "gang": 2},
    
    # GRUPO C: Zonas de Baja Incidencia ("Interior Seguro")
    "HERRERA":       {"pop": 125000,  "unemp": 3.5,  "gang": 1},
    "LOS SANTOS":    {"pop": 100000,  "unemp": 2.5,  "gang": 1}, # Provincia con menor tasa histórica
    "COCLÉ":         {"pop": 275000,  "unemp": 5.5,  "gang": 3},
    
    # GRUPO D: Zonas Fronterizas y Comarcales
    "DARIÉN":        {"pop": 65000,   "unemp": 4.0,  "gang": 5}, # Riesgo asociado a flujo migratorio
    "GUNA YALA":     {"pop": 48000,   "unemp": 4.5,  "gang": 3},
    "EMBERÁ":        {"pop": 15000,   "unemp": 3.5,  "gang": 2},
    "NGÄBE BUGLÉ":   {"pop": 230000,  "unemp": 4.0,  "gang": 2},
    
    # Mapeo de Variantes: Se incluyen nombres alternativos para asegurar coincidencia con el Excel
    "COMARCA GUNA YALA":     {"pop": 48000,  "unemp": 4.5, "gang": 3},
    "COMARCA KUNA YALA":     {"pop": 48000,  "unemp": 4.5, "gang": 3},
    "KUNA YALA":             {"pop": 48000,  "unemp": 4.5, "gang": 3},
    "COMARCA NGÄBE BUGLÉ":   {"pop": 230000, "unemp": 4.0, "gang": 2},
    "COMARCA EMBERÁ":        {"pop": 15000,  "unemp": 3.5, "gang": 2},
    "COMARCA EMBERÁ WOUNAAN":{"pop": 15000,  "unemp": 3.5, "gang": 2}
}

# ------------------------------------------------------------------------------
# 3. MOTOR DE GENERACIÓN DE TENDENCIAS (2017 - 2025)
# ------------------------------------------------------------------------------
data_rows = []
years = range(2017, 2026) # El rango incluye el año actual y proyecciones futuras

# Proceso Iterativo: Se recorre cada provincia para construir su historia año por año
for prov_real in provincias_reales:
    
    # Recuperación de parámetros base (Si no existe, usa valores por defecto neutrales)
    stats = datos_maestros.get(prov_real, {"pop": 50000, "unemp": 5.0, "gang": 1})
    
    for year in years:
        # --- ALGORITMO DE AJUSTE TEMPORAL ---
        
        # A. Proyección Demográfica:
        # Se aplica una tasa de crecimiento exponencial del 1.5% anual respecto al año base (2023).
        # Esto permite estimar poblaciones pasadas y futuras de manera coherente.
        diff_years = year - 2023
        pop_ajustada = int(stats['pop'] * (1.015 ** diff_years))
        
        # B. Modelado Económico (Simulación de Impacto COVID-19):
        # Se ajusta la tasa de desempleo base según eventos macroeconómicos históricos.
        unemp_base = stats['unemp']
        if year == 2020:
            unemp_ajustada = unemp_base * 1.8  # Crisis severa (Pandemia)
        elif year == 2021:
            unemp_ajustada = unemp_base * 1.4  # Recuperación económica lenta
        elif year < 2020:
            unemp_ajustada = unemp_base * 0.9  # Estabilidad pre-pandemia
        else:
            unemp_ajustada = unemp_base        # Retorno a niveles base actuales
            
        # C. Variabilidad Estocástica (Actividad Criminal):
        # Se introduce "ruido estadístico" (distribución normal) al índice de pandillas.
        # Esto evita que los datos sean líneas planas perfectas, simulando la fluctuación real del crimen.
        gang_noise = np.random.normal(0, 0.5) 
        # Se limita el rango entre 0 y 10 para mantener coherencia
        gang_ajustada = max(0, min(10, stats['gang'] + gang_noise))
        
        # Consolidación del registro anual
        data_rows.append({
            'AÑO': year,
            'PROVINCIA': prov_real, # Clave foránea para el Merge posterior
            'POBLACION_ESTIMADA': pop_ajustada,
            'TASA_DESEMPLEO': round(unemp_ajustada, 2),
            'INDICE_PANDILLAS': int(round(gang_ajustada))
        })

# ------------------------------------------------------------------------------
# 4. EXPORTACIÓN DE RESULTADOS
# ------------------------------------------------------------------------------
# Conversión de la lista de diccionarios a DataFrame y exportación a CSV
df_nuevo_contexto = pd.DataFrame(data_rows)
nombre_archivo = 'Datos_Contexto_Anual_MEJORADO.csv'
df_nuevo_contexto.to_csv(nombre_archivo, index=False)

print(f"\n[EXITO] [PROCESO COMPLETADO] Dataset de contexto generado exitosamente: {nombre_archivo}")
# Muestra de las primeras filas para verificación de calidad
print(df_nuevo_contexto.head())

[INFO] Entidades geográficas detectadas en el origen: 14
['SAN MIGUELITO' 'PANAMÁ OESTE' 'CHIRIQUÍ' 'COCLÉ' 'COLÓN' 'HERRERA'
 'VERAGUAS' 'PANAMÁ' 'BOCAS DEL TORO' 'DARIÉN' 'LOS SANTOS' 'NGÄBE BUGLÉ'
 'GUNA YALA' 'EMBERÁ WOUNAAN']

[EXITO] [PROCESO COMPLETADO] Dataset de contexto generado exitosamente: Datos_Contexto_Anual_MEJORADO.csv
    AÑO      PROVINCIA  POBLACION_ESTIMADA  TASA_DESEMPLEO  INDICE_PANDILLAS
0  2017  SAN MIGUELITO              347526            8.55                10
1  2018  SAN MIGUELITO              352738            8.55                10
2  2019  SAN MIGUELITO              358030            8.55                 9
3  2020  SAN MIGUELITO              363400           17.10                10
4  2021  SAN MIGUELITO              368851           13.30                10
